In [1]:
import heapq

import torch

In [22]:
class Beam:
    def __init__(
        self,
        beam_width: int,
        length_penalty: int = 1,
    ):
        """_summary_

        Args:
            beam_width (int): 束宽
            length_penalty (int, optional): 长度惩罚，默认为1，即不惩罚
            TODO:
                Coverage normalization：惩罚重复
                End of sentence normalization：抑制长句
        """
        self.heap = []
        self.beam_width = beam_width
        self.length_penalty = length_penalty

    def add(self, data):
        """
        CSC task is different from the generate task, the length of the result is fixed.
        """
        heapq.heappush(self.heap, data)

        if len(self.heap) > self.beam_width:
            heapq.heappop(self.heap)

    def pop_all(self):
        ret = []
        while self.heap:
            ret.append(heapq.heappop(self.heap))
        return ret

    def __iter__(self):
        """转化为可迭代对象，可使用其方法，比如max"""
        return iter(self.heap)

    def __len__(self):
        return len(self.heap)

    def __str__(self):
        return f"the paths is {[bm for bm in self]}"

In [23]:
def generate_random_tensor(shape):
    data = torch.randn(shape).type(torch.float16)
    return torch.softmax(data, dim=0)

In [51]:
def generate_with_beamsearch(self, beam_width, seq, batch_first=True):
    # add batch dimension
    if len(seq_shape) == 2:
        seq.unsqueeze_(~int(batch_first))
        batch_first = True

    if not batch_first:
        seq.transpose_(0, 1)
    batch_size = seq.shape[0]

    beam = Beam(beam_width)
    beam.add([0, [0]])

    ret_beam = []
    for batch in seq:
        for i, s in enumerate(batch):
            if s == -1:
                ret_beam.append(beam)
                break

            prefixs = beam.pop_all()
            for prob, path in prefixs:
                # TODO: according to the prefix sequence to generate the probility of the next token
                next_tokens = generate_random_tensor([50, 1])
                probs, inds = next_tokens.topk(beam_width, dim=0)
                print(probs)
                for i, cur_prob in enumerate(probs):
                    beam.add(
                        [prob - (torch.log(cur_prob).item()), path + [inds[i].item()]]
                    )
    return ret_beam

In [ ]:
def beam_search_generate(
    self,
    beam_width: int,
    sequence,
    length_penalty: int = 1,
    bos_token_id: int = 101,
    pad_token_id: int = 0,
    eos_token_id: int = 102,
    batch_first: bool = True,
):

    if not batch_first:
        sequence = sequence.transpose(0, 1)
    batch_size = sequence.shape[0]

    # beamers = [Beamer(beam_width=beam_width) for _ in range(batch_size)]
    beam = Beam()

In [176]:
a = tokenizer(sentence, return_tensors="pt")

In [177]:
a

{'input_ids': tensor([[ 101, 2769, 3221, 1920, 2358, 1520,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [178]:
aa = a["input_ids"]

In [179]:
aa

tensor([[ 101, 2769, 3221, 1920, 2358, 1520,  102]])

In [186]:
aa[:, :5] = torch.tensor([1, 2, 3, 4, 5])

In [187]:
aa

tensor([[   1,    2,    3,    4,    5, 1520,  102]])

In [188]:
t = [5, 4, 2, 3, 1]

In [194]:
torch.cat((torch.tensor(t).unsqueeze(), torch.tensor(t)))

TypeError: unsqueeze() missing 1 required positional arguments: "dim"

In [199]:
torch.cat((torch.tensor(t).unsqueeze(0), aa[:, 5:]), dim=1)

tensor([[   5,    4,    2,    3,    1, 1520,  102]])

In [448]:
import heapq

import torch.nn


class Beamer:
    def __init__(
        self,
        beam_width: int,
    ):
        """just a container for beam search

        Args:
            beam_width (int): _description_
        """
        self.heap = []
        self.beam_width = beam_width

    def add(self, item: tuple):
        """_summary_

        Args:
            item (tuple): _description_
        """
        heapq.heappush(self.heap, item)

        if len(self) > self.beam_width:
            heapq.heappop(self.heap)

    def pop_all(self):
        ret_beam = []
        while len(self) > 0:
            ret_beam.append(heapq.heappop(self.heap))

        return ret_beam

    def __len__(self):
        return len(self.heap)

    def __iter__(self):
        return iter(self.heap)


def beam_search_generate(
    self,
    beam_width: int,
    sequence,
    length_penalty: int = 1,
    bos_token_id: int = 101,
    pad_token_id: int = 0,
    eos_token_id: int = 102,
    batch_first: bool = False,
    device: str = "cuda",
):

    # if batch_first:
    #     sequence = sequence.transpose(0, 1)
    # batch_size = sequence.shape[0]
    self.to(device)

    # beamers = [Beamer(beam_width=beam_width) for _ in range(batch_size)]
    beamer = Beamer(beam_width=beam_width)
    beamer.add([0, [bos_token_id]])

    input_ids = sequence["input_ids"].unsqueeze(0).to(device)
    attention_mask = (
        sequence["attention_mask"].unsqueeze(0).to(device).type(torch.float)
    )
    labels = sequence["labels"].unsqueeze(0).to(device)

    with torch.no_grad():
        for i in range(input_ids.shape[1]):
            if input_ids[:, i].eq(eos_token_id).all():
                break

            prefixs = beamer.pop_all()
            for score, path in prefixs:
                cur_input_ids = torch.cat(
                    (torch.tensor(path).unsqueeze(0).to(device), input_ids[:, i + 1 :]),
                    dim=1,
                ).to(device)
                # print(cur_input_ids.shape, attention_mask.shape)
                output = self(cur_input_ids, src_mask=attention_mask)[:, -1, :]
                probs, inds = output.topk(beam_width, dim=-1)
                for j, cur_prob in enumerate(probs.squeeze()):
                    beamer.add(
                        [
                            score - (torch.log(cur_prob).item()),
                            path + [inds.squeeze()[j].item()],
                        ]
                    )
                    for bm in beamer:
                        print(bm)
    return beamer

In [449]:
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import BertModel, BertTokenizer

# Implemented by myself
from config import *
from data_processer import CSCDataset, split_torch_dataset
from models import CombineBertModel, DecoderBaseRNN, DecoderTransformer, Trainer

In [450]:
tokenizer = BertTokenizer.from_pretrained(checkpoint)

In [451]:
train_dataset = CSCDataset([SIGHAN_train_dir_err, SIGHAN_train_dir_corr], tokenizer)

test_dataset = CSCDataset([SIGHAN_train_dir_err14, SIGHAN_train_dir_corr14], tokenizer)

preprocessing sighan dataset: 2339it [00:00, 1045447.26it/s]
preprocessing sighan dataset: 100%|██████████████████████████████████████████| 2339/2339 [00:00<00:00, 25350069.91it/s]


共2339句，共73264字，最长的句子有171字


preprocessing sighan dataset: 3437it [00:00, 1143477.66it/s]
preprocessing sighan dataset: 100%|███████████████████████████████████████████| 3437/3437 [00:00<00:00, 1139590.74it/s]

共3437句，共170330字，最长的句子有258字


In [452]:
train_data, dev_data = split_torch_dataset(train_dataset, 0.3)

train_data_loader = DataLoader(train_data, num_workers=4, shuffle=True, batch_size=16)

In [453]:
test = train_dataset[0]
test

{'input_ids': tensor([ 101,  852, 3221, 2769,  679, 5543, 1343, 1347, 1217, 8024, 1728, 4158,
         2769, 3300,  671, 7953,  752, 2658, 7350, 8013,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0, 

In [454]:
setattr(CombineBertModel, "beam_search_generate", beam_search_generate)

In [455]:
# tokenizer = BertTokenizer.from_pretrained(checkpoint)
encoder_model = BertModel.from_pretrained(checkpoint)

# The Hyperparameters can be defined in config.py
hidden_size = 1024
num_layers = 2

decoder_model = DecoderBaseRNN(
    model=nn.LSTM,
    input_size=encoder_model.config.hidden_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
)

model = CombineBertModel(encoder_model=encoder_model, decoder_model=decoder_model).to(
    device
)

In [456]:
beam = model.beam_search_generate(beam_width=2, sequence=test)

[2.075861692428589, [101, 13327]]
[2.075861692428589, [101, 13327]]
[2.123257875442505, [101, 14251]]
[4.222711801528931, [101, 13327, 13327]]
[4.222711801528931, [101, 13327, 13327]]
[4.226269721984863, [101, 13327, 4296]]
[4.226269721984863, [101, 13327, 4296]]
[4.2656073570251465, [101, 14251, 13327]]
[4.2656073570251465, [101, 14251, 13327]]
[4.273269176483154, [101, 14251, 4296]]
[6.352656841278076, [101, 14251, 13327, 13327]]
[6.352656841278076, [101, 14251, 13327, 13327]]
[6.3621132373809814, [101, 14251, 13327, 15957]]
[6.35575795173645, [101, 14251, 4296, 13327]]
[6.3621132373809814, [101, 14251, 13327, 15957]]
[6.3621132373809814, [101, 14251, 13327, 15957]]
[6.378638505935669, [101, 14251, 4296, 15957]]
[8.440070867538452, [101, 14251, 13327, 15957, 13327]]
[8.440070867538452, [101, 14251, 13327, 15957, 13327]]
[8.444730043411255, [101, 14251, 13327, 15957, 15957]]
[8.444730043411255, [101, 14251, 13327, 15957, 15957]]
[8.459105968475342, [101, 14251, 4296, 15957, 15957]]
[8

KeyboardInterrupt: 